# West Nile Virus Prediction Model and Cost Analysis (Part 1/3)

## Problem Statement

The deadly, mosquito-borne West Nile Virus has long infested the City of Chicago. It is imperative to develop an accurate predictive model that could identify the key contributing factors that leads to proliferation of the virus. The model and the insights drawn could help the authority to predict future outbreaks of West Nile Virus and thus effectively allocate resources to mitigate it.

Logistic Regression, Naive Bayes, Extra Trees and XGboost models will be used to model the data and Accuracy and Recall score will be used to evaluate and find the best model for production and prediction.

# 1 Initialization

In [2]:
# Import Vanilla Libraries
import pandas as pd
import numpy as np

# Geospatial Libraries
import geopandas as gpd
from shapely.geometry import Point

# Pandas Settings
pd.set_option('max_columns', 999)

# 2 Weather Data

In [3]:
# Reading Weather Data, converting 'M' & '-' to Nan
weather = pd.read_csv('./assets/weather.csv', parse_dates=['Date'], na_values=['M','-'])

In [4]:
weather.shape

(2944, 22)

In [5]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67.0,14.0,51,56.0,0.0,2.0,448.0,1849.0,,0.0,NaN,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68.0,NaN,51,57.0,0.0,3.0,NaN,NaN,,NaN,NaN,NaN,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51.0,-3.0,42,47.0,14.0,0.0,447.0,1850.0,BR,0.0,NaN,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52.0,NaN,42,47.0,13.0,0.0,NaN,NaN,BR HZ,NaN,NaN,NaN,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56.0,2.0,40,48.0,9.0,0.0,446.0,1851.0,,0.0,NaN,0.0,0.00,29.39,30.12,11.7,7,11.9


In [6]:
weather.columns

Index(['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth',
       'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel',
       'ResultSpeed', 'ResultDir', 'AvgSpeed'],
      dtype='object')

In [7]:
weather.dtypes

Station                 int64
Date           datetime64[ns]
Tmax                    int64
Tmin                    int64
Tavg                  float64
Depart                float64
DewPoint                int64
WetBulb               float64
Heat                  float64
Cool                  float64
Sunrise               float64
Sunset                float64
CodeSum                object
Depth                 float64
Water1                float64
SnowFall               object
PrecipTotal            object
StnPressure           float64
SeaLevel              float64
ResultSpeed           float64
ResultDir               int64
AvgSpeed              float64
dtype: object

## 2.1 Data Type Issue

From the output above, we can see that all columns are numerical except for CodeSum, SnowFall and PrecipTotal. CodeSum is the code for weather condition at a given day, so it is sensible that it is not numerical. However, the other 2 columns are ought to be numerical. Let's look into each of the 2 columns to find out if there is any invalid value that is causing it to be categorical.

### 2.1.1 SnowFall & PrecipTotal

In [8]:
# Unique values of SnowFall
weather.SnowFall.unique()

array(['0.0', nan, '  T', '0.1'], dtype=object)

In [9]:
# Unique values of PrecipTotal
weather.PrecipTotal.unique()

array(['0.00', '  T', '0.13', '0.02', '0.38', '0.60', '0.14', '0.07',
       '0.11', '0.09', '1.01', '0.28', '0.04', '0.08', '0.01', '0.53',
       '0.19', '0.21', '0.32', '0.39', '0.31', '0.42', '0.27', '0.16',
       '0.58', '0.93', '0.05', '0.34', '0.15', '0.35', nan, '0.40',
       '0.66', '0.30', '0.24', '0.43', '1.55', '0.92', '0.89', '0.17',
       '0.03', '1.43', '0.97', '0.26', '1.31', '0.06', '0.46', '0.29',
       '0.23', '0.41', '0.45', '0.83', '1.33', '0.91', '0.48', '0.37',
       '0.88', '2.35', '1.96', '0.20', '0.25', '0.18', '0.67', '0.36',
       '0.33', '1.28', '0.74', '0.76', '0.71', '0.95', '1.46', '0.12',
       '0.52', '0.64', '0.22', '1.24', '0.72', '0.73', '0.65', '1.61',
       '1.22', '0.50', '1.05', '2.43', '0.59', '2.90', '2.68', '1.23',
       '0.62', '6.64', '3.07', '1.44', '1.75', '0.82', '0.80', '0.86',
       '0.63', '0.55', '1.03', '0.70', '1.73', '1.38', '0.44', '1.14',
       '1.07', '3.97', '0.87', '0.78', '1.12', '0.68', '0.10', '0.61',
       '0.

Here we see that '  T' are present in both the columns. Checking through the data dictionary document shows that the code 'T' actually represents 'Trace' amount of either Snow or Precipitation. We can easily replace the value with 0 since it was only traced amount that was not measurable by the sensory equipment.

In [10]:
# Replacing SnowFall 'T' with 0
weather['SnowFall'] = weather['SnowFall'].replace('  T', 0).astype(float)

# Replacing PrecipTotal 'T' with 0
weather['PrecipTotal'] = weather['PrecipTotal'].replace('  T', 0).astype(float)

In [11]:
# Sanity check on the data type after rectification
weather.dtypes

Station                 int64
Date           datetime64[ns]
Tmax                    int64
Tmin                    int64
Tavg                  float64
Depart                float64
DewPoint                int64
WetBulb               float64
Heat                  float64
Cool                  float64
Sunrise               float64
Sunset                float64
CodeSum                object
Depth                 float64
Water1                float64
SnowFall              float64
PrecipTotal           float64
StnPressure           float64
SeaLevel              float64
ResultSpeed           float64
ResultDir               int64
AvgSpeed              float64
dtype: object

Now our data type are in a good shape. We can proceed to perform some missing data imputation.

## 2.2 Missing Data Imputation

### 2.2.1 Data Exploration

We have noticed that the dataframe is actually made up of daily weather report from 2 different weather station in Chicago. So ultimately, we need to find a way to merge 2 stations into 1. But we have to do a sanity check on the value counts to confirm that they are indeed equal-sized.

In [12]:
# Sanity check on Station 1 and 2 value counts
weather.Station.value_counts()

1    1472
2    1472
Name: Station, dtype: int64

In [13]:
weather.groupby('Station').get_group(1).isnull().sum()

Station           0
Date              0
Tmax              0
Tmin              0
Tavg              0
Depart            0
DewPoint          0
WetBulb           3
Heat              0
Cool              0
Sunrise           0
Sunset            0
CodeSum           0
Depth             0
Water1         1472
SnowFall          0
PrecipTotal       0
StnPressure       2
SeaLevel          5
ResultSpeed       0
ResultDir         0
AvgSpeed          0
dtype: int64

In [14]:
weather.groupby('Station').get_group(2).isnull().sum()

Station           0
Date              0
Tmax              0
Tmin              0
Tavg             11
Depart         1472
DewPoint          0
WetBulb           1
Heat             11
Cool             11
Sunrise        1472
Sunset         1472
CodeSum           0
Depth          1472
Water1         1472
SnowFall       1472
PrecipTotal       2
StnPressure       2
SeaLevel          4
ResultSpeed       0
ResultDir         0
AvgSpeed          3
dtype: int64

From the missing value above, we can see that Water1 is totally missing from the dataframe. So we may just drop it straight away.
However, there are many columns that are totally missing in Station 2, but not Station 1. This is might be because Station 2 was not equipped with the necessary sensory equipment to capture the data. For this case, we simply need to perform a forward filling to fill up any missing data with the previous row.
So all missing data from Station 2 will be the same as the one from Station 1 on the same day. While some missing data in Station 1 will be acquired from the previous day, where it is more sensible than the mean/median.

In [15]:
# Dropping 'Water1' from the dataframe
weather.drop('Water1', axis=1, inplace=True)

In [16]:
# Forward filling to replace all missing value at Station 2 with Station 1 data
weather.fillna(method='ffill', inplace=True)

### 2.2.2 CodeSum

Next up, we need to deal with the only categorical data, **CodeSum**. From the first 5 rows of the data, we can see that there are rows with empty string. Apparently only days with weather condition will be recorded in the dataset, hence, empty string actually means that it is a weather-wise uneventful day. We can replace all empty string with 'NA' to indicate that.

In [17]:
# Checking on the composition of CodeSum
weather.CodeSum.head()

0         
1         
2       BR
3    BR HZ
4         
Name: CodeSum, dtype: object

In [18]:
# Replace all empty string with 'NA'
weather.CodeSum.replace(' ', 'NA', inplace=True)

In [19]:
# Sanity check on the composition again
weather.CodeSum.head()

0       NA
1       NA
2       BR
3    BR HZ
4       NA
Name: CodeSum, dtype: object

The **CodeSum** data would not be useful without proper encoding. We need to a label encoding to create columns for each weather condition. However, we can see that 2 or more weather conditions can happen in a day, so we would need to split each of the row into a list and label-encode everyone of them.

In [20]:
# Double for-loop to perform label encoding if the code is not 'NA'
for row, codes in enumerate(weather.CodeSum.str.split()):
    for code in codes:
        if code!='NA':
            weather.loc[row, code] = 1

# Dropping the original CodeSum column
weather.drop('CodeSum', axis=1, inplace=True)

# Fill up all missing value with 0 (or negative)
weather.loc[:, 'BR':] = weather.loc[:, 'BR':].fillna(0).astype(int)

## 2.3 Merging of Station 1 and 2 data

Next up, we need to merge the data from Station 1 and 2 by the average of two so that we only have 1 data row per day for the later merging with our core data.

In [21]:
# Merging Station 1 and 2 by groupby function to obtain the average value
cleaned_weather = weather.groupby('Date').mean().drop('Station', axis=1).reset_index()

However, sometimes a weather condition is only picked up by a single Weather Station. Hence, getting the average between the 2 stations will result in getting a 0.5. All the 0.5 should be rounded up to be 1 because as long as it is picked up by one weather station, it should be considered 'present' or 1.

In [22]:
# Replace all 0.5 in weather condition with 1
cleaned_weather.loc[:, 'BR':] = cleaned_weather.loc[:, 'BR':].replace(0.5, 1).astype(int)

In [23]:
# Final correction of Sunrise and Sunset datatype
cleaned_weather['Sunrise'] = cleaned_weather['Sunrise'].astype(int)
cleaned_weather['Sunset'] = cleaned_weather['Sunset'].astype(int)

## 2.4 Final Check and Export

In [24]:
# Sanity check on our cleaned weather data
cleaned_weather

,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,Depth,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,BR,HZ,RA,TSRA,VCTS,FU,TS,DZ,BCFG,FG+,MIFG,FG,SQ,SN,VCFG,GR
0,2007-05-01,83.5,51.0,67.5,14.0,51.0,56.5,0.0,2.5,448,1849,0.0,0.0,0.000,29.140,29.820,2.20,26.0,9.40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2007-05-02,59.5,42.5,51.5,-3.0,42.0,47.0,13.5,0.0,447,1850,0.0,0.0,0.000,29.410,30.085,13.15,3.0,13.40,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2007-05-03,66.5,47.0,57.0,2.0,40.0,49.0,8.0,0.0,446,1851,0.0,0.0,0.000,29.425,30.120,12.30,6.5,12.55,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2007-05-04,72.0,50.0,58.0,4.0,41.5,50.0,7.0,0.0,444,1852,0.0,0.0,0.000,29.335,30.045,10.25,7.5,10.60,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2007-05-05,66.0,53.5,60.0,5.0,38.5,49.5,5.0,0.0,443,1853,0.0,0.0,0.000,29.430,30.095,11.45,7.0,11.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,2014-10-27,78.0,52.5,65.5,16.0,51.5,58.5,0.5,1.0,618,1653,0.0,0.0,0.010,28.960,29.665,12.35,19.0,13.25,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1468,2014-10-28,67.0,46.5,57.0,10.0,39.0,47.5,8.0,0.0,619,1651,0.0,0.0,0.015,29.190,29.850,14.40,26.0,15.10,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1469,2014-10-29,49.0,38.0,44.0,-4.0,33.0,41.0,21.0,0.0,620,1650,0.0,0.0,0.000,29.390,30.065,9.00,29.0,9.45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1470,2014-10-30,52.0,34.5,43.5,-4.0,34.5,41.0,21.5,0.0,622,1649,0.0,0.0,0.000,29.375,30.095,5.50,23.5,6.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
# Export of cleaned data
cleaned_weather.to_csv('./assets/cleaned_weather.csv', index=False)

# 3 Spray Data

## 3.1 Spraying Data Encoding

In [26]:
# Reading all the remaining dataset
train = pd.read_csv('./assets/train.csv', parse_dates=['Date'])
test = pd.read_csv('./assets/test.csv', parse_dates=['Date'])
spray = pd.read_csv('./assets/spray.csv', parse_dates=['Date'])

In [27]:
# Remove duplicates in trainset
train.drop_duplicates(inplace=True)

In [28]:
# Function convert spraying data to a binary column

def spraying_calc(core_df, radius=100, period=14):
    # Import datetime library
    from datetime import timedelta
    
    lat_1 = core_df['Latitude']
    lon_1 = core_df['Longitude']
    check_date = core_df['Date']
    
    # Calculate spray_date given a effective period as args
    spray_date = check_date - timedelta(days=period)
    
    # Create shortlist of spraying data with date info
    spray_shortlist = spray[(spray['Date']<=check_date) & (spray['Date']>spray_date)]
    
    # Loop through all spraying activity during the effective period
    for idx in spray_shortlist.index:
        lat_2 = spray_shortlist.loc[idx, 'Latitude']
        lon_2 = spray_shortlist.loc[idx, 'Longitude']
        
        # Calculate Euclidean distance between coordinates (and convert to meters)
        distance = ((lat_1 - lat_2)**2 + (lon_1 - lon_2)**2) ** 0.5 * 100_000
        
        # Return 1 if distance shorter than effective radius
        if distance <= radius:
            return 1
        
    # Return 0 if none found
    return 0

In [29]:
train['IsSprayed'] = train.apply(spraying_calc, axis=1)

# Number of 1 in 'IsSprayed' in train.csv
print("Number of Sprayed Cases in trainset: ", train['IsSprayed'].sum())

Number of Sprayed Cases in trainset:  47


In [30]:
test['IsSprayed'] = test.apply(spraying_calc, axis=1)

# Number of 1 in 'IsSprayed' in test.csv
print("Number of Sprayed Cases in testset: ", test['IsSprayed'].sum())

Number of Sprayed Cases in testset:  0


## 3.2 Data Merging

In [31]:
# Merging of trainset with weather data
train = pd.merge(train, cleaned_weather, how = 'left', on='Date')

# Merging of testset with weather data
test = pd.merge(test, cleaned_weather, how = 'left', on='Date')

# 4 Preliminary Feature Engineering

## 4.1 Neighbourhood Encoding

Next up, we will encode all the data entries coordinates into their respective neighbourhood. The neighbourhood boundaries was obtained from the official City of Chicago data archive [link here](https://data.cityofchicago.org/api/geospatial/bbvz-uum9?method=export&format=GeoJSON). It was originally read and processed using the GeoPandas (gpd) library and its **gpd.read_file** function, however due to some known issue of incompatibility with the other common modules, the code is commented out. Instead, the geopandas dataframe was exported externally into a csv file for the alternative to read-in using both shapely and geopandas **gpd.GeoDataFrame** function (that has so far been working alright).

In [ ]:
### ORINGAL METHOD - GEOPANDAS (GPD) ###
### UNCOMMENT IT TO USE THE ORIGINAL ###

# # Read geojson file from Chicago City Official Archive
# neighbourhood = gpd.read_file('https://data.cityofchicago.org/api/geospatial/bbvz-uum9?method=export&format=GeoJSON')

# # Functions to label each row with their respective neighbourhood name
# def get_neighbourhood(row):
#     for boundary, neigh, in zip(neighbourhood.geometry, neighbourhood.pri_neigh):
#         coor = Point(row['Longitude'], row['Latitude'])
#         if coor.within(boundary):
#             return neigh
#     return 'Others'

In [38]:
###   ALTERNATIVES - KNOWN ISSUE   ###
### IN CASE OF GPD INCOMPATIBILITY ###

from shapely import wkt

neighbourhood = pd.read_csv('./assets/raw_neighbourhood.csv')
neighbourhood['geometry'] = neighbourhood['geometry'].apply(wkt.loads)
neighbourhood = gpd.GeoDataFrame(neighbourhood, geometry = 'geometry')

def get_neighbourhood(row):
    for boundary, neigh, in zip(neighbourhood.geometry, neighbourhood.pri_neigh):
        coor = Point(row['Longitude'], row['Latitude'])
        if coor.within(boundary):
            return neigh
    return 'Others'

The train, test and spraying data will all be encoded with their respective neighbourhood data. Apart from that, the centroid of each neighbourhood will be recorded and exported into a separate dataset for EDA purpose in the next notebook in the following cells:

In [39]:
# Get Neighbourhood for trainset
train['Neighbourhood'] = train.apply(get_neighbourhood, axis=1)

# Get Neighbourhood for testset
test['Neighbourhood'] = test.apply(get_neighbourhood, axis=1)

In [40]:
# Get Neighbourhood for spray data
spray['Neighbourhood'] = spray.apply(get_neighbourhood, axis=1)

# Export spray data for EDA purpose
spray.to_csv('./assets/spray_with_neigh.csv', index=False)

In [41]:
# Export neighbourhood data for EDA purpose
neighbourhood['lat']=neighbourhood.geometry.centroid.y
neighbourhood['lon']=neighbourhood.geometry.centroid.x

neighbourhood = neighbourhood[['pri_neigh', 'shape_area', 'lat', 'lon']]

neighbourhood.to_csv('./assets/neighbourhood.csv', index=False)

## 4.2 Weather Data

Sunrise and Sunset are now both in the integer format, which is not a good representation of time, instead we will be converting them into float values based on its hour.

In [42]:
# Convert Sunrise to Numerical
train.Sunrise = (train.Sunrise/100).astype(int) + ((train.Sunrise/100)%1 * 100/60)
test.Sunrise = (test.Sunrise/100).astype(int) + ((test.Sunrise/100)%1 * 100/60)

# Convert Sunset to Numerical
train.Sunset = (train.Sunset/100).astype(int) + ((train.Sunset/100)%1 * 100/60)
test.Sunset = (test.Sunset/100).astype(int) + ((test.Sunset/100)%1 * 100/60)

## 4.3 Neighbourhood Average

Instead of using the nominal category of neighbourhood name, we have decided on converting the neighbourhood information into numerical data. The dimensionality of our feature in the modelling could be reduced by this conversion. So rather than one-hot encoding all of the neighbourhood name into more than 50 different columns that are relatively sparse in nature, we will convert each of the neighbourhood into its neighbourhood-wise average value. This could retain the information of each neighbourhood, but without the high and sparse data structure in the model at later stage.

In [43]:
# Feature engineering to obtain a neighbourhood-average value of WnvPresent
ave_wnv = pd.DataFrame(train.groupby('Neighbourhood').mean()['WnvPresent']).reset_index()

# Renaming the columns
ave_wnv.columns = ['Neighbourhood', 'Neighbourhood_Ave_Wnv']

In [44]:
# Merge trainset with the WNV-average value
train = pd.merge(train, ave_wnv, how='left', on='Neighbourhood')

# Merge testset with the WNV-average value
test = pd.merge(test, ave_wnv, how='left', on='Neighbourhood')

# Impute any missing neighbourhood from trainset with the mean
test['Neighbourhood_Ave_Wnv'].fillna(test['Neighbourhood_Ave_Wnv'].mean(), inplace=True)

## 4.4 Mosquitoes Species

Apparently some of the mosquitos species has zero record of being WNV-positive, we can safely assume that these mosquito species are not suitable as a carrier of WNV. Hence again, to reduce the dimensionality, we will combined all the non-carrier species into a single category of '0'. The string 0 is used so that it will always be dropped at first during one-hot encoding.

In [45]:
# Title-transform the species column so it is more readable as a column
train['Species'] = train['Species'].str.title()
test['Species'] = test['Species'].str.title()

In [46]:
# Check on the WNV risk of each mosquito species
train.groupby('Species').mean()['WnvPresent']

Species
Culex Erraticus           0.000000
Culex Pipiens             0.092899
Culex Pipiens/Restuans    0.055046
Culex Restuans            0.018338
Culex Salinarius          0.000000
Culex Tarsalis            0.000000
Culex Territans           0.000000
Name: WnvPresent, dtype: float64

In [47]:
# Replacing mosquito species with zero WNV risk with '0'
# So it would be dropped automatically during one-hot encoding
replace_dict = {
    'Culex Erraticus': 0,
    'Culex Salinarius': 0,
    'Culex Tarsalis': 0,
    'Culex Territans': 0
}

# Apply the mapping dict on both train and test set
train['Species'] = train['Species'].replace(replace_dict)
test['Species'] = test['Species'].replace(replace_dict)

## 4.5 Single-value Column Removal

There are some columns that has one unique value across the train dataset. These columns would provide zero contribution or information to the modelling attempt. They shall be dropped.

In [48]:
# Dropping unnecessary columns
train.drop(['Address', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'AddressAccuracy'], axis=1, inplace=True)

In [49]:
# Sanity check on the trainset
train.head()

,Date,Species,Latitude,Longitude,NumMosquitos,WnvPresent,IsSprayed,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,Depth,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,BR,HZ,RA,TSRA,VCTS,FU,TS,DZ,BCFG,FG+,MIFG,FG,SQ,SN,VCFG,GR,Neighbourhood,Neighbourhood_Ave_Wnv
0,2007-05-29,Culex Pipiens/Restuans,41.954690,-87.800991,1,0,0,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,4.35,19.283333,0.0,0.0,0.0,29.415,30.1,5.8,17.0,6.95,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Dunning,0.084833
1,2007-05-29,Culex Restuans,41.954690,-87.800991,1,0,0,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,4.35,19.283333,0.0,0.0,0.0,29.415,30.1,5.8,17.0,6.95,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Dunning,0.084833
2,2007-05-29,Culex Restuans,41.994991,-87.769279,1,0,0,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,4.35,19.283333,0.0,0.0,0.0,29.415,30.1,5.8,17.0,6.95,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Sauganash,Forest Glen",0.046584
3,2007-05-29,Culex Pipiens/Restuans,41.974089,-87.824812,1,0,0,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,4.35,19.283333,0.0,0.0,0.0,29.415,30.1,5.8,17.0,6.95,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Norwood Park,0.128205
4,2007-05-29,Culex Restuans,41.974089,-87.824812,4,0,0,88.0,62.5,75.5,10.0,58.5,65.5,0.0,10.5,4.35,19.283333,0.0,0.0,0.0,29.415,30.1,5.8,17.0,6.95,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Norwood Park,0.128205


In [50]:
# Get the number of classes in each columns
unique_counts = train.select_dtypes(include='number').apply(pd.Series.value_counts).notna().sum()

# Discard any column with only a single class in the entire trainset
drop_cols = unique_counts[unique_counts<2].index

# Apply the selected column on the final trainset and testset
train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)

# 5 Final Data Export

In [52]:
# Export the cleaned and engineered dataset to CSV
train.to_csv('./assets/train_final.csv', index=False)
test.to_csv('./assets/test_final.csv', index=False)

In [53]:
# Sanity Check on the exported Data
train = pd.read_csv('./assets/train_final.csv', parse_dates=['Date'])

train.shape

(9693, 33)

In [54]:
train.sample(10)

,Date,Species,Latitude,Longitude,NumMosquitos,WnvPresent,IsSprayed,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,BR,HZ,RA,TSRA,VCTS,TS,DZ,FG,Neighbourhood,Neighbourhood_Ave_Wnv
5853,2011-07-11,Culex Restuans,41.957799,-87.930995,39,0,0,89.0,70.5,80.0,7.0,70.5,74.0,0.0,15.0,4.433333,19.466667,0.260,29.150,29.815,5.40,24.5,8.25,1,1,0,1,0,0,0,0,Others,0.053512
4931,2009-09-03,Culex Restuans,41.798697,-87.736812,3,0,0,75.0,56.5,66.0,-2.0,54.5,59.5,0.0,1.0,5.316667,18.400000,0.000,29.450,30.150,4.95,5.5,5.95,1,0,0,0,0,0,0,0,Archer Heights,0.063380
7680,2013-06-27,Culex Restuans,41.891118,-87.654491,1,0,0,90.5,65.0,78.0,6.0,66.0,69.5,0.0,13.0,4.300000,19.516667,0.755,29.045,29.720,6.65,29.0,7.00,1,1,0,0,0,0,0,0,West Town,0.005128
30,2007-06-05,Culex Pipiens/Restuans,41.921600,-87.666455,1,0,0,63.5,49.0,56.5,-9.0,47.5,51.5,8.5,0.0,4.283333,19.383333,0.345,29.130,29.785,5.70,4.0,7.85,1,0,1,0,0,0,0,0,Lincoln Park,0.049751
9281,2013-09-06,Culex Pipiens,41.778748,-87.586427,1,1,0,85.5,59.5,73.0,5.0,56.5,63.5,0.0,8.0,5.366667,18.316667,0.000,29.410,30.110,4.50,18.5,6.00,0,0,0,0,0,0,0,0,Woodlawn,0.031447
9548,2013-09-19,Culex Pipiens/Restuans,41.766202,-87.562889,1,0,0,84.0,69.5,77.0,14.0,67.0,69.5,0.0,12.0,5.600000,17.916667,0.420,29.175,29.870,8.05,16.0,10.30,1,1,1,1,0,1,0,1,South Shore,0.012195
91,2007-06-26,Culex Pipiens/Restuans,41.921600,-87.666455,1,0,0,91.5,71.5,81.5,10.0,69.0,72.0,0.0,16.5,4.300000,19.516667,0.080,29.420,30.100,6.70,21.5,7.60,1,1,0,1,1,0,0,0,Lincoln Park,0.049751
3779,2009-06-26,Culex Pipiens/Restuans,41.773215,-87.600880,1,0,0,86.0,70.5,78.5,7.0,60.5,70.0,0.0,13.5,4.300000,19.516667,0.000,29.175,29.840,6.40,4.0,8.10,0,0,0,0,0,0,0,0,Woodlawn,0.031447
6204,2011-07-25,Culex Restuans,41.987280,-87.666066,1,0,0,89.0,73.0,81.5,7.0,66.5,71.0,0.0,16.5,4.633333,19.300000,0.005,29.225,29.900,2.75,33.5,6.30,0,0,0,0,0,0,0,0,Edgewater,0.071429
407,2007-07-11,Culex Restuans,41.794781,-87.615989,18,0,0,77.0,62.5,70.0,-3.0,51.0,59.0,0.0,5.0,4.433333,19.466667,0.000,29.255,29.925,8.95,30.0,11.05,0,0,0,0,0,0,0,0,Washington Park,0.000000


In [55]:
test = pd.read_csv('./assets/test_final.csv', parse_dates=['Date'])

test.sample(5)

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,IsSprayed,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,BR,HZ,RA,TSRA,VCTS,TS,DZ,FG,Neighbourhood,Neighbourhood_Ave_Wnv
25568,25569,2008-09-03,"2900 West 85th Street, Chicago, IL 60652, USA",Culex Pipiens,29,W 85TH ST,T237,"2900 W 85TH ST, Chicago, IL",41.738903,-87.695443,8,0,77.0,67.0,72.5,4.0,61.0,64.5,0.0,7.5,5.316667,18.400000,0.01,29.325,30.005,10.50,1.0,11.25,0,0,0,1,0,0,0,0,Ashburn,0.090164
50563,50564,2010-08-06,"6000 West Roscoe Street, Chicago, IL 60634, USA",Culex Restuans,60,W ROSCOE ST,T005,"6000 W ROSCOE ST, Chicago, IL",41.942114,-87.776385,8,0,82.0,64.0,73.0,0.0,58.5,64.0,0.0,8.0,4.833333,19.100000,0.00,29.250,29.920,6.10,29.0,6.40,0,0,0,0,0,0,0,0,Dunning,0.084833
52635,52636,2010-08-19,"3600 North Pittsburgh Avenue, Chicago, IL 6063...",Culex Pipiens/Restuans,36,N PITTSBURGH AVE,T011,"3600 N PITTSBURGH AVE, Chicago, IL",41.944869,-87.832763,8,0,87.5,67.5,77.5,6.0,64.0,69.0,0.0,12.5,5.050000,18.800000,0.00,29.210,29.900,4.60,21.0,5.25,0,0,0,0,0,0,0,0,Dunning,0.084833
1200,1201,2008-06-11,"2100 North Cannon Drive, Chicago, IL 60614, USA",Culex Pipiens/Restuans,21,N CANNON DR,T054C,"2100 N CANNON DR, Chicago, IL",41.925652,-87.633590,8,0,86.0,63.5,75.0,7.0,55.5,64.0,0.0,10.0,4.266667,19.433333,0.00,29.310,29.980,9.15,18.0,10.20,0,0,0,0,0,0,0,0,Lincoln Park,0.049751
16172,16173,2008-08-05,"6100 North Lemont Avenue, Chicago, IL 60646, USA",0,61,N LEMONT AVE,T012,"6100 N LEMONT AVE, Chicago, IL",41.991429,-87.747113,8,0,87.5,70.0,79.0,5.0,68.0,71.5,0.0,14.0,4.816667,19.116667,0.04,29.270,29.920,4.55,31.0,6.85,1,1,0,0,0,0,0,0,"Sauganash,Forest Glen",0.046584


Now the dataset is cleaned and engineered, the project will be continued on the second part of the notebook. Exploratory analysis and multiple modelling approaches will be conducted in the second part.